# GET VALVE

In [1]:
from math import sin, cos, radians, pi
import cadquery as cq
from cadquery import Workplane
from cadquery import exporters
from collections import namedtuple
from jupyter_cadquery.viewer.client import show, show_object
from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=True, timeit=False)
cv = open_viewer("Examples", cad_width=640, height=480, glass=True)
enable_replay(False, False)
show_object = replay

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


# PARTS

In [5]:
def flanges(*,flange_c, pipe_c, gap_left, gap_right, shield_c,thck = 5, left_side=False,  bolts_num = 8, bolts_d=5):
    '''
    GENERATES TWO FLANGES WITH PIPE AND SHIELD
    WORKS BOTH FOR LEFT AND RIGHT SIDE
    '''
    # try:
    #GET DIAMETER
    flange_d = flange_c/pi
    pipe_d = pipe_c/pi
    shield_d = shield_c/pi

    #ERROR CATCHING
    if pipe_d >= flange_d:
        raise ValueError('"pipe_c" cannot be greater than or equal to "flange_c"')
    elif (flange_d - pipe_d) <= bolts_d:
        raise ValueError('Holes do not fit this flange. Make holes smaller ("bolts_d") or increase "flange_d"')

    #BASE CYLINDER
    base = Workplane("XZ").cylinder(thck,flange_d)

    #CENTER HOLE
    front_face = Workplane("XZ").workplane(offset=thck).moveTo(0,0)
    sketch = front_face.circle(pipe_d).extrude(-thck*2)
    result = base.cut(sketch)

    #SMALL HOLES
    ofst = pipe_d + ((flange_d-pipe_d)/2)
    for i in range(bolts_num):
        angle = i * (360/bolts_num)
        x = ofst * cos(radians(angle)) # MATH MODULE
        y = ofst * sin(radians(angle))
        front_face = Workplane("XZ").workplane(offset=thck).moveTo(x,y)
        sketch = front_face.circle(bolts_d).extrude(-thck*2)
        result = result.cut(sketch)

    #MIRRORING THE FLANGES
    result = result.translate(result.val().BoundingBox().center.multiply(-1))
    mirXZ_neg = result.mirror(mirrorPlane="XZ", basePointVector=(0, (thck/2)+0.1, 0))
    result = result.union(mirXZ_neg,breakpoint)


    #PIPE WITH THE SHIELD
    face, coef, gap = "<Y", 1, gap_right
    if left_side:
        face, coef, gap = ">Y", -1, gap_left

    result = (result
                  .faces(face)
                  .circle(pipe_d)
                  .extrude(gap/coef)
                  .faces(face)
                  .circle(shield_d)
                  .extrude(700/coef)
                 )
    return result
    # except Exception as e:
    #     print(f"Flange function: {e}")
        
    #result.val().exportStep('res.stp')
    #show_object(result)
    

In [6]:
def body(*,valve_l=1100,flange_c=2820,pipe_c=1950,valve_c=800,thck=20, act_h=50, actuator_gap = 200):
    #Get diameter
    flange_d = flange_c/pi
    pipe_d = pipe_c/pi
    valve_d=valve_c/pi
    print(valve_d)
    
    if valve_d >= valve_l:
        raise ValueError('"valve_c" cannot be greater than or equal to "valve_l"') 
    
    #Get length of body excluding the flange thickness
    total = valve_l - (thck*4) # flange is always 50mm
    
    side = (total-valve_d)/2

    #creating the body of valve
    bdy = (cq.Workplane("XZ")
          .move(0,0)
          .circle(pipe_d)
          .workplane(offset=-side)
          .circle(flange_d)
          .loft(combine=True)
          .faces(">Y")
          .circle(flange_d)
          .extrude(-valve_d)
          .faces(">Y")
          .circle(flange_d)
          .workplane(offset=side)
          .circle(pipe_d)
          .loft(combine=True)
          )
    
    #adding the actuator to the top of body
    actuator = (cq.Workplane("XY")
                .move(0,(total/2))
                .circle(valve_d/2)
                .extrude(flange_d+(flange_d/2))
               )
    
    bdy = bdy.union(actuator)
    
    
    return bdy

Assembly

In [15]:
def build(right_flange, left_flange, bdy, path_step='valve.step'):
    valve = (cq.Assembly()
             .add(right_flange
                  ,loc=cq.Location((0, 0, 0), (0, 1, 0), 0)
                  , name="right"
                  , color=cq.Color("lightgray")
                  )
             .add(left_flange
                  , name="left"
                  , color=cq.Color("lightgray")
                  )
             .add(bdy
                  , name="body"
                  , color=cq.Color("lightgray")
                  )
             )

    (valve
     .constrain("right@faces@>Y", "body@faces@<Y", "Plane")
     .constrain("body@faces@>Y", "left@faces@<Y", "Plane")
     )
    
    
    valve.solve()
    
    show(valve)

    # user = input("Generated without errors. Save this valve as STEP?(y\\n) ")
    # if user == 'y':
    #     valve.save(path)
    #     print(f"Saved to directory: {path}")
    # print('\nProgram finished')

# CALCULATE

In [21]:
def get_jacket_dxf(*, valve_l=1100, flange_c=2820, valve_c=800, shield_c=2750
                   , gap_left=350, gap_right=310, thck=30, path_sketch='sheet.dxf'
                   , X_offset=50, Y_ofst = 400, hole_offset=200):

    valve_d = round(valve_c / pi,3)
    center = valve_l - (2 * thck) # THE LENGTH OF THE CENTER SIDE (BODY OF VALVE)
    
    # OFFSETS
    valve_d += hole_offset # Increasing the hole
    
    gap_left += X_offset # (X axis) increasing the left gap
    gap_right += X_offset # (X axis) increasing the right gap
    
    flange_c += Y_ofst # (Y axis) increasing circumference depending on a thickness of jacket (MIDDLE SIDE)
    shield_c += Y_ofst # (Y axis) increasing circumference depending on a thickness of jacket (TOP SIDES)
    
    # FOR FINDING THE HOLE COORDINATES
    center_hole = (center - valve_d) / 2

    # VARIABLES FOR ARCS
    # Firts point of the ThreePointArc is defined by the last drawn line
    arc1_p2 = (valve_d/2, -(center_hole+(valve_d/2)))
    arc1_p3 = (0, -(center_hole+valve_d))
    arc2_p2 = ((flange_c-(valve_d/2)), -(center/2))
    arc2_p3 = (flange_c, -center_hole)

    # CREATING SKETCH
    jack = (cq.Workplane("XY")
            .moveTo(0.0) # STARTING POINT
            .lineTo(flange_c, 0.0)  # SECTION 1
            .lineTo(shield_c + (flange_c - shield_c) / 2, gap_left)  # SECTION 1
            .lineTo((flange_c - shield_c) / 2, gap_left)  # SECTION 1
            .lineTo(0.0, 0.0)  # SECTION 1
            .lineTo(0, -center_hole)  # SECTION 2 (part 1)
            .threePointArc(arc1_p2, arc1_p3) # SECTION 2 (part 1)
            .lineTo(0, -center)  # SECTION 2 (part 1)
            .lineTo(flange_c, -center)  # SECTION 3
            .moveTo(0, -center)  # MOVING THE WORKING POSITION
            .lineTo((flange_c - shield_c) / 2, -(center + gap_right))  # SECTION 3
            .lineTo(shield_c + (flange_c - shield_c) / 2, -(center + gap_right))  # SECTION 3
            .lineTo(flange_c, -center)  # SECTION 3
            .lineTo(flange_c, -(center_hole+valve_d)) # SECTION 2 (part 2)
            .threePointArc(arc2_p2, arc2_p3) # SECTION 2 (part 2)
            .lineTo(flange_c, 0) # SECTION 2 (part 2)
            .close()
            )
    
    show_object(jack)

    cq.exporters.exportDXF(jack, path_sketch)

In [17]:
'''
 flange_c - circumference of the flange
 pipe_c - circumference of the pipe
 shield_c - circumference of the shield
 valve_l - total length of the valve
 valve_c - circumference of the valve(top side)
 gap_left - gap btw flange and shield(left)
 gap_right - gap btw flange and shield(right)
 actuator_gap - distance btw actuator and flange
 actuator_type - electric, manual or other
 actuator_l - length of the actuator (actuator_type: other)
'''

def get_valve_stp(*, flange_c, pipe_c, shield_c, valve_l, valve_c, gap_left, gap_right, actuator_gap, thck, bolts_num=8,
                  bolts_d=5, actuator_type='electric', path_step='valve.step'):
    r_flange = flanges(flange_c=flange_c
                       , pipe_c=pipe_c
                       , gap_left=gap_left
                       , gap_right=gap_right
                       , shield_c=shield_c
                       , left_side=False
                       , thck=thck
                       , bolts_num=bolts_num
                       , bolts_d=bolts_d
                       )

    l_flange = flanges(flange_c=flange_c
                       , pipe_c=pipe_c
                       , gap_left=gap_left
                       , gap_right=gap_right
                       , shield_c=shield_c
                       , left_side=True
                       , thck=thck
                       , bolts_num=bolts_num
                       , bolts_d=bolts_d
                       )

    bdy = body(valve_l=valve_l
               , flange_c=flange_c
               , pipe_c=pipe_c
               , valve_c=valve_c
               , thck=thck
               )

    build(r_flange, l_flange, bdy, path_step)

# MAIN

In [25]:
#MAIN
if __name__=='__main__':
    # VARIABLES FOR BOTH 3D MODEL AND JACKET
    flange_c = 500  # circumference of the flange (REQUIRED)
    valve_l = 280  # length of the valve (REQUIRED)
    shield_c = 460  # circumference of the shield (REQUIRED)
    pipe_c = 240  # circumference of the pipe (REQUIRED)
    valve_c = 500  # circumference of the valve (top side) (REQUIRED)
    gap_left = 80  # gap btw flange and shield (left) (REQUIRED)
    gap_right = 50  # gap btw flange and shield (right) (REQUIRED)
    thck = 20  # thickness of the flange (REQUIRED)
    actuator_gap = 200  # gap from the edge of actuator and outter flange (OPTIONAL)
    path_step='H:\\Desktop\\valve2.step'  # (3D model) path where the file will be saved. Default: 'valve.step'
    # bolts_num= # amount of holes in a flange, by default 8 (OPTIONAL)
    bolts_d = 10 # hole diameter, by default 5 (OPTIONAL)
    
    # VARIABLES FOR 2D SKETCH ONLY
    X_offset = 100
    Y_ofst = 100
    hole_offset = 20
    path_sketch = 'H:\\Desktop\\sheet.dxf'
    
    # GETTING A 3D MODEL OF VALVE (output is .step file)
    get_valve_stp(flange_c=flange_c
                  , valve_l=valve_l
                  , shield_c=shield_c
                  , pipe_c=pipe_c
                  , valve_c=valve_c
                  , gap_left=gap_left
                  , gap_right=gap_right
                  , thck=thck
                  , actuator_gap=actuator_gap
                  # , bolts_num=bolts_num
                  , bolts_d=bolts_d
                  , path_step=path_step
                  )
    
    # GETTING A 2D SKETCH OF THE JACKET FOR A VALVE (output is .dxf file)
    get_jacket_dxf(flange_c=flange_c
                   , valve_l=valve_l
                   , shield_c=shield_c
                   , valve_c=valve_c
                   , gap_left=gap_left
                   , gap_right=gap_right
                   , thck=thck
                   , X_offset=X_offset
                   , Y_ofst=Y_ofst
                   , hole_offset=hole_offset
                   , path_sketch=path_sketch
                  )

159.15494309189535
100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.31s
Use the multi select box below to select one or more steps you want to examine


# TESTING/TEMPLATES

In [6]:
result = (cq.Workplane("front").box(4.0, 4.0, 0.25).faces(">Z").circle(1.5)
          .workplane(offset=3.0).rect(0.75, 0.5).loft(combine=True))



show_object(result)

Use the multi select box below to select one or more steps you want to examine


In [ ]:
#CHOOSING A FACE

# res = (cq
#        .Workplane("XY") # choose the starting face where
#        .circle(30)
#        .extrude(20)
#        .faces(">Z") # THE TOP FACE OF THE OBJECT
#        .circle(20)
#        .extrude(20)
#        .faces("<Z") # THE BOTTOM FACE
#        .rect(20,20)
#        .extrude(-20)
#        .faces("<Z")
#        .circle(20)
#        .extrude(-20)
#       )

#LOFT
# res = (cq
#        .Workplane("front")
#        .circle(20)
#        .workplane(offset=10)
#        .rect(20,20)
#        .loft(combine=True)
#       )
show_object(res)

In [ ]:
result0 = (cq.Workplane("XY")
           .moveTo(10, 0)
           .lineTo(5, 0)
           .threePointArc((3.9393, 0.4393), (3.5, 1.5))
           .threePointArc((3.0607, 2.5607), (2, 3))
           .lineTo(1.5, 3)
           .threePointArc((0.4393, 3.4393), (0, 4.5))
           .lineTo(0, 13.5)
           .threePointArc((0.4393, 14.5607), (1.5, 15))
           .lineTo(28, 15)
           .lineTo(28, 13.5)
           .lineTo(24, 13.5)
           .lineTo(24, 11.5)
           .lineTo(27, 11.5)
           .lineTo(27, 10)
           .lineTo(22, 10)
           .lineTo(22, 13.2)
           .lineTo(14.5, 13.2)
           .lineTo(14.5, 10)
           .lineTo(12.5, 10)
           .lineTo(12.5, 13.2)
           .lineTo(5.5, 13.2)
           .lineTo(5.5, 2)
           .threePointArc((5.793, 1.293), (6.5, 1))
           .lineTo(10, 1)
           .close())
result = result0.extrude(100)

result = result.rotate((0, 0, 0), (1, 0, 0), 90)

result = result.translate(result.val().BoundingBox().center.multiply(-1))

mirXY_neg = result.mirror(mirrorPlane="XY", basePointVector=(0, 0, -30))
mirXY_pos = result.mirror(mirrorPlane="XY", basePointVector=(0, 0, 30))
mirZY_neg = result.mirror(mirrorPlane="ZY", basePointVector=(-30, 0, 0))
mirZY_pos = result.mirror(mirrorPlane="ZY", basePointVector=(30, 0, 0))

result = result.union(mirXY_neg).union(mirXY_pos).union(mirZY_neg).union(mirZY_pos)

show_object(result)

In [ ]:
#MIRROR EXAMPLE

result = flanges(flange_c=2820,pipe_c=1950,gap_left=350, gap_right=310, shield_c=2750)
    
result = result.translate(result.val().BoundingBox().center.multiply(-1))

mirXY_neg = result.mirror(mirrorPlane="XZ", basePointVector=(0, 1100, 0))

result = result.union(mirXY_neg)

show_object(result)

In [ ]:
!pip list

In [5]:
c = cq.Workplane("XY").box(1, 1, 1).color(255, 0, 0)

AttributeError: 'Workplane' object has no attribute 'color'

In [4]:
print(cq.__version__)

2.2.dev570+g60fdbea


In [9]:
box = cq.Workplane("XY").box(1, 1, 1).shapes()[0]
red_box = box.color(cadquery.Color.RED)

AttributeError: 'Workplane' object has no attribute 'shapes'

In [11]:
# Create the individual parts
box = cq.Workplane("XY").box(1, 1, 1)
cylinder = cq.Workplane("XY").cylinder(1, 2)

# Translate the cylinder to be on top of the box
translated_cylinder = cylinder.translate((0, 0, 1))

# Combine the box and cylinder into an assembly
assembly = box.union(translated_cylinder)

show_object(assembly)

Use the multi select box below to select one or more steps you want to examine


In [34]:

# Define the parts to be assembled
def part1():
    return cq.Workplane("XY").box(1, 1, 1)

def part2():
    return cq.Workplane("XY").box(1, 2, 1)

# Assemble the parts into an Assembly object
assembly = cq.Assembly().add(part1(), name="part1").add(part2(), name="part2")

# Constrain the parts relative to each other
assembly.constrain("part1@faces@>Z", "part2@faces@<Z", "Axis")

# Solve the constraints to determine the positions of the parts
assembly.solve()

# Render the assembly
assembly.save('assemb.step')

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.01s


In [40]:
b1 = cq.Workplane("XY"

b2 = 

b1 = b1.translate((0,-1.5,0))

#mirXY_neg = result.mirror(mirrorPlane="XZ", basePointVector=(0, 1100, 0))

result = b1.union(b2)


In [41]:
show_object(result)

Use the multi select box below to select one or more steps you want to examine


In [28]:
f = flanges(pipe_c=110,flange_c=370,shield_c=340,gap_left=30,gap_right=50)

show_object(f)

Use the multi select box below to select one or more steps you want to examine


In [53]:
result = (cq.Workplane(origin = (20,0,0))
    .circle(2)
    .revolve(180, (-20,0,0),(-20,-1,0))
    .center(-20,0)
    .workplane()
    .rect(20,4)
    .extrude("next")
)

show_object(result)

Use the multi select box below to select one or more steps you want to examine


In [9]:
r1 = 79/2
r2 = 80/2

circles = pi*(r1**2) + pi*(r2**2)

total_sa = ((pi*r1 + pi*r2)*(((r1-r2)**2)+(100**2))**0.5) + pi*((r1**2) + (r2**2)) - circles

print(round(total_sa))

24976


In [18]:
line = (cq.Workplane("XY")
        .center(0,0)
        .lineTo(0,20)
        .center(0,0)
        .vLine(10)
       )

show_object(line)

Use the multi select box below to select one or more steps you want to examine


In [18]:
(L, H, W, t) = (100.0, 20.0, 20.0, 1.0)
pts = [
    (0, H/2.0),
    (W/2.0, H/2.0),
    (W/2.0, (H/2.0 - t)),
    (t/2.0, (H/2.0 - t)),
    (t/2.0, (t - H/2.0)),
    (W/2.0, (t - H/2.0)),
    # (W/2.0, H/-2.0),
    # (0, H/-2.0)
]

result = cq.Workplane("XY").polyline(pts).edges()

print(result)

show_object(result)

Use the multi select box below to select one or more steps you want to examine


In [92]:
s = Workplane().lineTo(1,0).lineTo(1,1).close().extrude(0.2)

show_object(s)

Use the multi select box below to select one or more steps you want to examine


In [9]:
# Define the three points that will define the arc
p1 = (0, -392.611)
p2 = (127.389, -520.00)
p3 = (0, -647.389)

# Create the arc by passing the three points to the arc() method
arc = cq.Edge.makeThreePointArc(p1, p2, p3).close()

arc1 = cq.Workplane("XY").add(arc)

show(arc1)

In [ ]:
result0 = (cq.Workplane("XY")
           .moveTo(10, 0)
           .lineTo(5, 0)
           .threePointArc((3.9393, 0.4393), (3.5, 1.5))
           .threePointArc((3.0607, 2.5607), (2, 3))
           .lineTo(1.5, 3)
           .threePointArc((0.4393, 3.4393), (0, 4.5))
           .lineTo(0, 13.5)
           .threePointArc((0.4393, 14.5607), (1.5, 15))
           .lineTo(28, 15)
           .lineTo(28, 13.5)
           .lineTo(24, 13.5)
           .lineTo(24, 11.5)
           .lineTo(27, 11.5)
           .lineTo(27, 10)
           .lineTo(22, 10)
           .lineTo(22, 13.2)
           .lineTo(14.5, 13.2)
           .lineTo(14.5, 10)
           .lineTo(12.5, 10)
           .lineTo(12.5, 13.2)
           .lineTo(5.5, 13.2)
           .lineTo(5.5, 2)
           .threePointArc((5.793, 1.293), (6.5, 1))
           .lineTo(10, 1)
           .close())

show_object(result0)

In [18]:
test = (cq.Workplane("XY")
        .moveTo(0,0)
        .lineTo(0,-300)
        .threePointArc((25, -300), (0, -400))
        .lineTo(0,-700)
        .lineTo(300, -300)
        .lineTo(0,0)
        .close())

show_object(test)

Use the multi select box below to select one or more steps you want to examine
